# Functional Programming

In Python, everything you interact with is an object. From functions to immutable data types, they are all objects. So, when I say functional programming, I mean from the perspective of not creating custom objects through classes that we define ourselves. That will be covered in the next chapter.

## Operators

The first thing I want to talk about are the operators used to perform math and logic.

### Math Operators

The basic mathematical operators of Python are exactly what you expect: add (+), subtract (-), multiply (*), and divide (/). These are demonstrated below.

In [42]:
# I know the below prints aren't PEP 8, but it is cleaner for these
x = 1024                                      ;print("x = {}".format(x))
y = 2

x = y + 1                                     ;print("x = {}".format(x))
x = y - 20                                    ;print("x = {}".format(x))
x = y * 56                                    ;print("x = {}".format(x))
x = y / 2                                     ;print("x = {}".format(x))
x = -y                                        ;print("x = {}".format(x))

x = 1024
x = 3
x = -18
x = 112
x = 1.0
x = -2


Notice how division, even when it results in an integer, is a float type.

In addition to these basic operators, three more are provided: floor division (//), modulus (%), and exponentiation (*)

In [43]:
x = 4 ** 2                                    ;print("x = {}".format(x))
x = 21 // 2                                   ;print("x = {}".format(x))
x = 4.5 // 2                                  ;print("x = {}".format(x))
x = 4.0 // 2                                  ;print("x = {}".format(x))
x = 4 // 2.0                                  ;print("x = {}".format(x))
x = 21 % 6                                    ;print("x = {}".format(x))

x = 16
x = 10
x = 2.0
x = 2.0
x = 2.0
x = 3


Floor division clears the remainder and keeps the number as an integer if both the divisor and dividend are integers, else the output is a float.

Modulus returns the remainder of the division.

### Bitwise Operators

These operators are incredibly useful for engineers when checking status bits. I have come across people using the bin() function to convert an integer to a string of 1s and 0s and then checking individual characters. This is both time and memory inefficient compared to bit shifting and anding. Before we go into use cases, let's cover the basics, however.

In [44]:
# The 'and' (&) operator only lets bits through that 
# are the same in both numbers
x = 0b110011 & 0b101101                   ;print("x = 0b{:b}".format(x))
# The 'or' operator (|) lets bits pass if they exist in either number
x = 0b110011 | 0b101101                   ;print("x = 0b{:b}".format(x))
# The 'exclusive or' operator (^) only lets bits through that exist
# in one or the other number, but not both
x = 0b110011 ^ 0b101101                   ;print("x = 0b{:b}".format(x))
# Bit shift right (>>) shifts the bits to the right. 
# Bits that 'fall off' are discarded.
x = 0b1101101 >> 3                        ;print("x = 0b{:b}".format(x))
# Bit shift left (>>) shifts the bits to the left. 
# Zeros are added from the right.
x = 0b1101101 << 3                        ;print("x = 0b{:b}".format(x))

x = 0b100001
x = 0b111111
x = 0b11110
x = 0b1101
x = 0b1101101000


Now, lets cover two simple use cases. We have a status word coming off the engine. The 5th bit (with the 1s bit being the 1st) tells us if the engine is overheating, so we want to check it out. The 3rd and 4th bits are used to determine the state of a nozzle. They follow this pattern:

| 3 | 4 | Meaning                  |
|---|---|:-------------------------|
| 0 | 0 | Nozzle if off            |
| 1 | 0 | Nozzle has low flow      |
| 0 | 1 | Nozzle has high flow     |
| 1 | 1 | Nozzle has malfunctioned |

Now let's suppose our boss wants to know if the Nozzle has flow and if it has malfunctioned. He also wants to know if the engine overheated. This should be easy enough, so lets start coding.

In [45]:
status = 0b10100  # High flow, overheating

# To check if overheating has happened, lets check the 5th bit
# To do this, we can and with 0b10000 to isolate the 5th bit
overheat = status & 0b10000      ;print("overheat = {}".format(bool(overheat)))
# Now, lets check if there is flow. We need to determine if the 3rd or 
# 4th bit is set but not both, for this we can use exclusive or
bit3 = status & 0b00100                  ;print("bit3 = {}".format(bool(bit3)))
bit4 = status & 0b01000                  ;print("bit4 = {}".format(bool(bit4)))
flow = bit3 ^ bit4                       ;print("flow = {}".format(bool(flow)))
# Finally, the malfunction occurs if bit3 and bit4 are set
malfunction = bit3 & bit4  ;print("malfunction = {}".format(bool(malfunction)))

overheat = True
bit3 = True
bit4 = False
flow = True
malfunction = False


There is one more bitwise operator that I haven't covered. This is the complement (~) operator. In most languages, this one inverts the values of all of the bits and is referred to as 'not.' However in Python, integers don't have a fixed length and are always signed, so the not operator works a little differently than one might expect:

In [46]:
it = 0b101011                          ;print("it = 0b{:b}".format(it))
not_it = ~it                           ;print("not_it = 0b{:b}".format(not_it))

it = 0b101011
not_it = 0b-101100


If you've come from another language, you probably expected not_it to equal 0b0101001, so what is going on here? It turns out, that since integers have no fixed length and are always signed, the ~ operator is actually equivalent to -x-1, where x + ~x = -1. This is proved below.

In [47]:
x = 0b1010110
print("x = {}".format(x))
print("~x = {}".format(~x))
print("-x-1 = {}".format(-x-1))
print("x + ~x = {}".format(x + ~x))

x = 86
~x = -87
-x-1 = -87
x + ~x = -1


If you just need to invert all of the bits, the best way to accomplish that is through a mask like so:

In [48]:
# Get the number of bits used in x: 7
# Bit shift up to that size number: 0b10000000
# Subtract 1 to create a full set of 1s of length 7: 0b1111111
# Parenthesis are important because of order of operations
mask = (1 << x.bit_length()) - 1         ;print("mask = 0b{:b}".format(mask))
# XOR will capture all of the oposite bits
# The ones set in the mask but not x
not_x = x ^ mask                         ;print("not_x = 0b{:b}".format(not_x))

mask = 0b1111111
not_x = 0b101001


#### *Side Note: Self Application Shorthand*

All symbol based operators have a shorthand form to apply the action to the current value. For example: x = x + 1 can also be written as x += 1. Below you can see many of them in action.

In [49]:
x = 1024                                      ;print("x = {}".format(x))
x += 1                                        ;print("x = {}".format(x))
x -= 10                                       ;print("x = {}".format(x))
x *= 2                                        ;print("x = {}".format(x))
x **= 3                                       ;print("x = {}".format(x))
x /= 9                                        ;print("x = {}".format(x))
x //= 2                                       ;print("x = {}".format(x))
x = int(x)  # Reset back to an integer
x &= 0b11101                                  ;print("x = {}".format(x))
x |= 0b00010                                  ;print("x = {}".format(x))
x ^= 0b10101                                  ;print("x = {}".format(x))
x >>= 2                                       ;print("x = {}".format(x))
x <<= 2                                       ;print("x = {}".format(x))

x = 1024
x = 1025
x = 1015
x = 2030
x = 8365427000
x = 929491888.8888888
x = 464745944.0
x = 24
x = 26
x = 15
x = 3
x = 12


### Comparison Operators

### Logical Operators 

### Order of Operations

Python follows the standard PEMDAS order of operations that you have come to expect from your calculators. However, bitwise, conditional, and logical operators also need to fit in somewhere. Bitwise operators are above conditional operators whereas logical operators sit below everything else. This is why you don't need parenthesis when doing logical chaining of expressions. The complete list can be found below. Operators on the same level are evaluated left to right.
- Parenthesis: ()
- Unary Operators (Negative, Bitwise Complement): -, ~
- Exponentiation: **
- Multiplication, Division: *, /, //, %
- Addition, Subtraction: +, -
- Bitwise Shift: <<, >>
- Bitwise AND: &
- Bitwise XOR: ^
- Bitwise OR: |
- Comparison: <, <=, >=, >
- Equivalence: ==, !=
- Identity Checking: is
- Contains: in
- Logical NOT: not
- Logical AND: and
- Logical OR: or

## Conditionals and Looping

### If Statement

### For Loop

### While Loop

## Functions

### Standard Functions

### Generators

### Decorators

### Argument Unpacking